In [ ]:
# Import required libraries
import heapq  # For priority queue implementation used in A* algorithm
import numpy as np  # For creating and manipulating the maze grid
import matplotlib.pyplot as plt  # For visualizing the maze and path
from matplotlib.colors import ListedColormap  # For custom color mapping in visualization

class MazeSolver:
    def __init__(self, width=10, height=10, obstacle_density=0.2):
        """Initialize the maze with given dimensions and obstacle density"""
        # Set maze dimensions
        self.width = width  # Width of the maze (number of columns)
        self.height = height  # Height of the maze (number of rows)
        
        # Create empty maze grid (0 = free space, 1 = obstacle)
        self.maze = np.zeros((height, width))
        
        # Generate random obstacles in the maze
        self._generate_maze(obstacle_density)
        
        # Set start position (top-left corner)
        self.start = (0, 0)
        # Set goal position (bottom-right corner)
        self.goal = (height-1, width-1)
        
        # Ensure start and goal positions are not blocked by obstacles
        self.maze[self.start] = 0  # Clear start position
        self.maze[self.goal] = 0  # Clear goal position
    
    def _generate_maze(self, obstacle_density):
        """Generate random obstacles in the maze based on given density"""
        # Loop through each cell in the maze
        for i in range(self.height):  # For each row
            for j in range(self.width):  # For each column
                # Randomly place obstacle based on density probability
                if np.random.random() < obstacle_density:
                    self.maze[i][j] = 1  # Mark as obstacle
    
    def heuristic(self, a, b):
        """Calculate Manhattan distance between two points (heuristic for A*)"""
        # Manhattan distance = sum of absolute differences of coordinates
        return abs(a[0] - b[0]) + abs(a[1] - b[1])
    
    def get_neighbors(self, pos):
        """Get all valid neighboring cells (up, down, left, right)"""
        neighbors = []  # List to store valid neighbors
        row, col = pos  # Current position coordinates
        
        # Check all 4 possible movement directions
        for dr, dc in [(-1, 0), (1, 0), (0, -1), (0, 1)]:  # (delta row, delta column)
            # Calculate neighbor position
            r, c = row + dr, col + dc
            
            # Check if neighbor is within maze boundaries and not an obstacle
            if (0 <= r < self.height and  # Check row bounds
                0 <= c < self.width and  # Check column bounds
                self.maze[r][c] == 0):  # Check if cell is free
                neighbors.append((r, c))  # Add valid neighbor to list
        
        return neighbors
    
    def solve(self):
        """Solve the maze using A* search algorithm"""
        # Priority queue for nodes to explore, ordered by f_score
        open_set = []
        # Add start node to open set with initial priority (f_score)
        heapq.heappush(open_set, (0, self.start))
        
        # Dictionary to keep track of optimal path (node -> came from)
        came_from = {}
        # Dictionary to store cost from start to each node (g_score)
        g_score = {self.start: 0}  # Start node has 0 cost
        # Dictionary to store estimated total cost (f_score = g_score + heuristic)
        f_score = {self.start: self.heuristic(self.start, self.goal)}
        
        # Set for quick lookup of nodes in open_set
        open_set_hash = {self.start}
        
        # Main A* algorithm loop
        while open_set:
            # Get node with lowest f_score from priority queue
            current = heapq.heappop(open_set)[1]
            # Remove from lookup set
            open_set_hash.remove(current)
            
            # Check if current node is the goal
            if current == self.goal:
                # Reconstruct path by backtracking from goal to start
                path = []
                while current in came_from:
                    path.append(current)  # Add current node to path
                    current = came_from[current]  # Move to previous node
                path.append(self.start)  # Add start node
                path.reverse()  # Reverse to get path from start to goal
                return path  # Return the solved path
            
            # Explore all valid neighbors of current node
            for neighbor in self.get_neighbors(current):
                # Calculate tentative g_score (current g_score + movement cost (1))
                tentative_g_score = g_score[current] + 1
                
                # If this path to neighbor is better than any previous one
                if (neighbor not in g_score or  # Neighbor not yet evaluated
                    tentative_g_score < g_score[neighbor]):  # Or better path found
                    
                    # Update path information
                    came_from[neighbor] = current  # Record best path to neighbor
                    g_score[neighbor] = tentative_g_score  # Update g_score
                    # Calculate f_score (g_score + heuristic)
                    f_score[neighbor] = tentative_g_score + self.heuristic(neighbor, self.goal)
                    
                    # If neighbor not already in open set
                    if neighbor not in open_set_hash:
                        # Add to open set with its f_score as priority
                        heapq.heappush(open_set, (f_score[neighbor], neighbor))
                        # Add to lookup set
                        open_set_hash.add(neighbor)
        
        # If open set is empty and goal not reached
        return None  # No path exists
    
    def visualize(self, path=None):
        """Visualize the maze with optional path"""
        # Define color map for visualization:
        # 0 = white (free), 1 = black (obstacle), 
        # 2 = green (start), 3 = red (goal), 4 = blue (path)
        cmap = ListedColormap(['white', 'black', 'green', 'red', 'blue'])
        
        # Create a copy of maze for display (to avoid modifying original)
        display_maze = np.copy(self.maze)
        
        # Mark start position (green)
        display_maze[self.start] = 2
        # Mark goal position (red)
        display_maze[self.goal] = 3
        
        # If path exists, mark it on the display maze (blue)
        if path:
            # Skip first (start) and last (goal) positions
            for (r, c) in path[1:-1]:
                display_maze[r][c] = 4  # Mark path cells as blue
        
        # Create figure for visualization
        plt.figure(figsize=(8, 8))
        # Display maze with custom color mapping
        plt.imshow(display_maze, cmap=cmap)
        
        # Add grid lines to better distinguish cells
        plt.grid(which='both', color='gray', linestyle='-', linewidth=0.5)
        # Remove axis ticks for cleaner look
        plt.xticks(np.arange(-0.5, self.width, 1), [])
        plt.yticks(np.arange(-0.5, self.height, 1), [])
        
        # Create legend elements
        legend_elements = [
            plt.Rectangle((0, 0), 1, 1, fc='white', label='Free'),
            plt.Rectangle((0, 0), 1, 1, fc='black', label='Obstacle'),
            plt.Rectangle((0, 0), 1, 1, fc='green', label='Start'),
            plt.Rectangle((0, 0), 1, 1, fc='red', label='Goal'),
        ]
        # Add path to legend if it exists
        if path:
            legend_elements.append(plt.Rectangle((0, 0), 1, 1, fc='blue', label='Path'))
        
        # Display legend outside the plot
        plt.legend(handles=legend_elements, bbox_to_anchor=(1.05, 1), loc='upper left')
        # Set plot title
        plt.title('Maze with A* Pathfinding')
        # Show the plot
        plt.show()

# Main execution
if __name__ == "__main__":
    # Create maze with specified dimensions and obstacle density
    maze = MazeSolver(width=15, height=15, obstacle_density=0.25)
    # Solve maze using A* algorithm
    path = maze.solve()

    # Print results
    if path:
        print("Path found with length:", len(path))
        print("Path coordinates:")
        # Print each step of the path
        for i, pos in enumerate(path):
            print(f"Step {i}: {pos}")
    else:
        print("No path found!")

    # Visualize the maze and path
    maze.visualize(path)